# Попытка деланья пробит модели с помощью Stan

## Загружаем библиотеки

In [1]:
#conda install pystan

In [31]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
import pystan
from tqdm.auto import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 8
%precision 4
plt.style.use('_classic_test_patch')

## Генерируем данные

Пусть будет 2 независимых переменных, одна из которых будет иметь нормальное распределение $\mathcal{N}(3, 4)$, другая -- равномерное $U(1, 3)$. Коэффициенты будут 2 (константа), -1 (для нормально распределенной случайной величины) и 0.5 (для равномерной случайной величины). Выборка будет состоять из 1000 объектов:

In [3]:
np.random.seed(999)

n = 1000
x_0 = np.ones(shape=n)
x_1 = np.random.normal(loc=3, scale=2, size=n)
x_2 = np.random.uniform(low=1, high=3, size=n)
X = np.vstack((x_0, x_1, x_2)).T

betas = np.array([2, -1, 0.5])
eps = np.random.normal(loc=0, scale=1, size=n) # предпосылка пробита

y = (X @ betas + eps > 0) * 1
print(f'Доля единиц в целевой переменной: {np.mean(y)}')

Доля единиц в целевой переменной: 0.508


## Модель

In [4]:
probit_data = {
             'n': n,
             'y': y,
             'x_1': x_1,
             'x_2': x_2
            }

In [19]:
%%time

model = pystan.stan(file = '/Users/markymark/Desktop/Статистика/py/probit_norm.stan', model_name='PriorNorm',
              data = probit_data, 
              chains = 1,
              iter = 2000)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL PriorNorm_09dfbcb4a3939d32350603f73a0f565f NOW.


CPU times: user 5.74 s, sys: 65.5 ms, total: 5.81 s
Wall time: 45.8 s


In [20]:
print(model)

Inference for Stan model: PriorNorm_09dfbcb4a3939d32350603f73a0f565f.
1 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=1000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
beta0   2.31    0.01   0.25   1.83   2.14    2.3   2.48   2.86    309    1.0
beta1  -0.96  2.7e-3   0.06  -1.07  -0.99  -0.96  -0.92  -0.85    444    1.0
beta2    0.3  5.0e-3    0.1    0.1   0.23    0.3   0.37   0.51    430    1.0
lp__  -330.3    0.07   1.31 -333.8 -330.9 -329.9 -329.3 -328.8    348    1.0

Samples were drawn using NUTS at Wed Jan 19 19:07:27 2022.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


Вроде получилось, теперь посчитаем метрики на 100 симуляциях, но сперва напишем функции, чтоб все было полаконичнее:

In [55]:
from sklearn.metrics import mean_absolute_percentage_error as mape, mean_squared_error as mse

params = ['beta0', 'beta1', 'beta2']

def get_metrics_for_coef(true: np.array, pred: np.array):
    MAPE = mape(true, pred)
    RMSE = mse(true, pred, squared=False)
    return MAPE, RMSE

def summary(way):
    results = {'beta0': {'MAPEs': [], 'RMSEs': []},
               'beta1': {'MAPEs': [], 'RMSEs': []},
               'beta2': {'MAPEs': [], 'RMSEs': []}
              }
    # строим модели
    for epoch in tqdm(range(100)):
        fit = pystan.stan(file = way,
              data = probit_data, 
              chains = 1,
              iter = 2000)
        # записываем результаты
        for i in range(len(betas)):
            pred_coef = fit.extract()[params[i]]
            true_coef = np.ones(shape=len(pred_coef)) * betas[i]
            MAPE, RMSE = get_metrics_for_coef(true_coef, pred_coef)
            results[params[i]]['MAPEs'].append(MAPE)
            results[params[i]]['RMSEs'].append(RMSE)
    # подсчитываем средние
    total = []
    for i in params:
        total.append([i, np.mean(results[i]['MAPEs']), np.mean(results[i]['RMSEs'])])
    return total
        

In [56]:
np.random.seed(999)
results_norm_prior = summary('/Users/markymark/Desktop/Статистика/py/probit_norm.stan')
results_norm_prior

  0%|          | 0/100 [00:00<?, ?it/s]

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09df

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09df

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09df

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09dfbcb4a3939d32350603f73a0f565f NOW.


[['beta0', 0.1543434296008626, 0.372188908088809],
 ['beta1', 0.059690760538910786, 0.07242940603460055],
 ['beta2', 0.3873377042823805, 0.2159392298344672]]

In [58]:
np.random.seed(999)
results_uniform_prior = summary('/Users/markymark/Desktop/Статистика/py/probit_uniform.stan')
results_norm_prior

  0%|          | 0/100 [00:00<?, ?it/s]

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0ead

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0ead

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0ead

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0eade79347dcfc7e053ec3c21aed3303 NOW.


[['beta0', 0.1543434296008626, 0.372188908088809],
 ['beta1', 0.059690760538910786, 0.07242940603460055],
 ['beta2', 0.3873377042823805, 0.2159392298344672]]